In [1]:
import numpy as np
from hw8_dataload import LFD_Data2
from svm_kernel import SVM_Poly

# HW 8
## Primal vs Dual Problem

- The SVM **primal** problem minimizes 0.5w<sup>T</sup>w subject to y<sub>n</sub>(w<sup>T</sup>x<sub>n</sub>+b)&ge;1 for n = 1,2,...,N
- Since w is a d-dimensional vector (corresponding to the dimension of x) and we can also vary b, the primal problem involves a quadratic programming problem with **d+1 variables**.

## Polynomial Kernels

- Implementing polynimal kernels with a soft-margin SVM using the given data set of handwritten digits from the US Postal Service Zip Code data set with extracted features digit, intensity, and symmetry. 
- The polynomial kernel K(x<sub>n</sub>, x<sub>m</sub>) = (1+x<sub>n</sub><sup>T</sup>x<sub>m</sub>)<sup>Q</sup>
- Training **two** types of binary classifiers:
    - one-vs-one (one digit class is +1, another is -1, rest are ignored)
    - one-vs-all (one digit class is +1, everything else is -1)

In [2]:
hw8_train = "hw8_train.dta"
hw8_test = "hw8_test.dta"
hw8_C = 0.01
hw8_Q = 2
hw8_data = LFD_Data2(hw8_train, hw8_test)

hw8_svmpoly = SVM_Poly(hw8_Q, hw8_C)

alphas_odd = np.array([])
alphas_even = np.array([])
for cur_num in range(10):
    #cur_num-vs-all
    hw8_data.set_filter([cur_num])
    hw8_svmpoly.train(hw8_data.get_X("train"), hw8_data.get_Y("train"))
    cur_error = hw8_svmpoly.calc_error(hw8_data.get_X("train"), hw8_data.get_Y("train"))
    cur_numalphas = hw8_svmpoly.num_alphas
    print("%d-vs-all binary classifier E_in: %f" % (cur_num, cur_error))
    if cur_num % 2 == 0:
        alphas_even = np.concatenate((alphas_even, [cur_numalphas]))
    else:
        alphas_odd = np.concatenate((alphas_odd, [cur_numalphas]))

aodd_sum = np.sum(alphas_odd)
aeven_sum = np.sum(alphas_even)
a_diff = abs(aodd_sum - aeven_sum)
print("Diff in number of sv's between odd and even: %d" % a_diff)
    
    

KeyboardInterrupt: 